This notebook generates a heat-map with the centers of the bounding boxes

In [ ]:
import json
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 16})

In [ ]:
INSTAN_PATH_VAL = "../data/datasets/COCO/annotations/instances_train2014.json"
NUMBER_OF_SECTORS = 16

In [ ]:
with open(INSTAN_PATH_VAL, "r") as json_file:
    coco = json.load(json_file)

In [ ]:
image_id_to_size = {}
for object_data in coco['images']:
    image_id = str(object_data['id'])
    image_id_to_size[image_id] = (object_data['width'], object_data['height'])

In [ ]:
objects = {}
for object_data in coco['annotations']:
    image_id = str(object_data['image_id'])
    bbox = object_data['bbox']
    WW, HH = image_id_to_size[image_id]
    bbox[0], bbox[1], bbox[2], bbox[3] = bbox[0] / WW, bbox[1] / HH, (bbox[0] + bbox[2]) / WW, (bbox[1] + bbox[3]) / HH
    # Centers
    bbox[0], bbox[1] = (bbox[0] + bbox[2]) * 0.5, (bbox[1] + bbox[3]) * 0.5
    if object_data['category_id'] in objects:
        objects[object_data['category_id']].append(bbox)
    else:
        objects[object_data['category_id']] = [bbox]

In [ ]:
category_to_name = {}
for object_data in coco['categories']:
    category_to_name[object_data['id']] = object_data['name']

In [ ]:
print(objects[2])

In [ ]:
class HeatMap():
    def __init__(self, number_of_categories, number_of_sectors, category_to_name):
        self.number_of_categories = number_of_categories
        self.number_of_sectors = number_of_sectors
        self.category_to_name = category_to_name
        self.number_of_objects_in_category = {}
        self.heat_maps = {}
        
        for i in range(number_of_categories):
            self.heat_maps[i] = np.array([[0]*(self.number_of_sectors) for i in range(self.number_of_sectors)])
    
    def print_heat_map(self, category_id):
        for i in self.heat_maps[category_id]:
            print(i)
    
    def add_element(self, category_id, x, y):
        # Lets suppose that x, y are in range [0, 1]
        x = x * self.number_of_sectors
        y = y * self.number_of_sectors
        pos_x = int(x) 
        pos_y = int(y)
        
        if pos_x == self.number_of_sectors:
            pos_x -= 1
        if pos_y == self.number_of_sectors:
            pos_y -= 1
            
        self.heat_maps[category_id][pos_y][pos_x] += 1
        if category_id in self.number_of_objects_in_category:
            self.number_of_objects_in_category[category_id] += 1
        else:
            self.number_of_objects_in_category[category_id] = 1
    
    def show_heat_map(self, category_id):
        axis = np.arange(0, 1, 1/(self.number_of_sectors))
        
        h_m = self.heat_maps[category_id]
        
        fig, ax = plt.subplots(figsize=(15,15), )
        # self.print_heat_map(category_id)
        im = ax.imshow(h_m, cmap = 'Greens')
        cbar = ax.figure.colorbar(im, ax=ax)
        cbar.ax.set_ylabel("Number of elements", rotation=-90, va="bottom")
        
        ax.set_xticks(np.arange(len(axis)))
        ax.set_yticks(np.arange(len(axis)))
        
        ax.set_xticklabels(axis)
        ax.set_yticklabels(axis)
        
        plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")
        
        for i in range(len(axis)):
            for j in range(len(axis)):
                text = ax.text(j, i, h_m[i, j],
                    ha="center", va="center", color="k")
        
        ax.set_title(self.category_to_name[category_id])
        fig.tight_layout()
        fig.savefig("./heat-map/" + str(category_id) + '.png', dpi=fig.dpi)
        plt.show()

In [ ]:
def add_category_to_heat_map(heat_map, objects, category_id):
    for bbox in objects[category_id]:
        heat_map.add_element(category_id, bbox[0], bbox[1])
    

In [ ]:
# create the heat map
heat_map = HeatMap(100, NUMBER_OF_SECTORS, category_to_name)

In [ ]:
# add all the categories
for key in objects.keys():
    add_category_to_heat_map(heat_map, objects, key)

In [ ]:
# kite x, y 
for bbox in objects[38]:
    print(bbox[0], bbox[1])

In [ ]:
heat_map.show_heat_map(38)

In [ ]:
heat_map.show_heat_map(1)

In [ ]:
for key in objects.keys():
    heat_map.show_heat_map(key)

Generate a gif with all the outputs

In [ ]:
import imageio
import glob
images = []
filenames = list(glob.glob("./heat-map/*"))

In [ ]:
print(filenames)
print(len(filenames))

In [ ]:
for filename in filenames:
    images.append(imageio.imread(filename))
imageio.mimsave('./class_heat-map.gif', images)